In [ ]:
!gdown --id 1eSo7XcOuW1BlLLt4YT5Vo-gK1FgR8xMU

In [ ]:
!unzip -q nature_12K.zip

In [ ]:
https://drive.google.com/file/d/1eSo7XcOuW1BlLLt4YT5Vo-gK1FgR8xMU/view?usp=sharing

In [ ]:
from torch.nn import Module, ReLU, Conv2d, Linear, MaxPool2d, LogSoftmax, NLLLoss, Dropout, BatchNorm2d, LeakyReLU, GELU, SELU, Mish
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch import flatten, float, no_grad
from torch.optim import Adam
import torch
import wandb
import math
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
PARAMETERS = {
    'data_augmentation': True,
    'batch_normalization': False,
    'filters': 64, # no. of filters in first layer
    'filter_org': 'half', # 'half', 'double'
    'dropout': 0.2,
    'activation': 'relu',
    'train_data_dir': "/kaggle/working/inaturalist_12K/train",
    'test_data_dir': "/kaggle/working/inaturalist_12K/val",
    'batch_size': 64,
    'learning_rate': 0.001,
    'epochs': 25,
    'dim': 256,
    'conv_kernel_size': 3,
    'dense_neurons': 1000
}

In [ ]:
CLASSES = {
    0:'Amphibia',
    1:'Animalia',
    2:'Arachnida',
    3:'Aves',
    4:'Fungi',
    5:'Insecta',
    6:'Mammalia',
    7:'Mollusca',
    8:'Plantae',
    9:'Reptilia'
}

In [ ]:
def get_data(param, type):
    if(type.lower() == 'train'):
        if param['data_augmentation']:
            transform = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.3),
                transforms.RandomRotation(degrees=12),
                transforms.Resize((param['dim'],param['dim'])),
                transforms.ToTensor(), 
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
            ])
        else:
            transform = transforms.Compose([
                transforms.Resize((param['dim'],param['dim'])),
                transforms.ToTensor(), 
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
            ])

        tdataset = datasets.ImageFolder(root=param['train_data_dir'], transform=transform)
        total = len(tdataset)
        train_sample = math.ceil(total*(0.8))
        val_sample = total-train_sample
        # print(total, train_sample, val_sample)
        train_dataset, validation_dataset = torch.utils.data.random_split(tdataset, [train_sample, val_sample])
        train_dataloader = DataLoader(train_dataset, batch_size=param['batch_size'], shuffle=True)
        validation_dataloader = DataLoader(validation_dataset, batch_size=param['batch_size'], shuffle=False)
        return train_dataloader, validation_dataloader
    
    else:
        transform = transforms.Compose([
            transforms.Resize((param['dim'],param['dim'])),
            transforms.ToTensor(), 
            transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
        ])
        test_dataset = datasets.ImageFolder(root=param['test_data_dir'], transform=transform)
        test_dataloader = DataLoader(test_dataset, batch_size=param['batch_size'])
        return test_dataloader

    

In [ ]:
class CNN(Module):
    def __init__(self, param):
        super(CNN, self).__init__()
        self.param=param
        self.data_augmentation = param['data_augmentation']
        self.dropout = param['dropout']
        self.act = self.getActivation(param['activation'])
        self.filters = self.filter_logic(param['filters'], param['filter_org'])
        self.conv_ks = param['conv_kernel_size']
        self.dim = param['dim']
        self.bn = param['batch_normalization']
        self.dense_neurons = param['dense_neurons']


        ####### Layer 1 #######
        curr_dim = self.dim
        self.conv1 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=3, out_channels=self.filters[0])
        curr_dim -= (self.conv_ks-1)
        self.act1 = self.act
        if(self.bn): self.bn1 = BatchNorm2d(self.filters[0])
        self.pool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout1 = Dropout(p=self.dropout)

        ####### Layer 2 #######
        self.conv2 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[0], out_channels=self.filters[1])
        curr_dim -= (self.conv_ks-1)
        self.act2 = self.act
        if(self.bn): self.bn2 = BatchNorm2d(self.filters[1])
        self.pool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout2 = Dropout(p=self.dropout)

        ####### Layer 3 #######
        self.conv3 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[1], out_channels=self.filters[2])
        curr_dim -= (self.conv_ks-1)
        self.act3 = self.act
        if(self.bn): self.bn3 = BatchNorm2d(self.filters[2])
        self.pool3 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout3 = Dropout(p=self.dropout)

        ####### Layer 4 #######
        self.conv4 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[2], out_channels=self.filters[3])
        curr_dim -= (self.conv_ks-1)
        self.act4 = self.act
        if(self.bn): self.bn4 = BatchNorm2d(self.filters[3])
        self.pool4 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout4 = Dropout(p=self.dropout)


        ####### Layer 5 #######
        self.conv5 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[3], out_channels=self.filters[4])
        curr_dim -= (self.conv_ks-1)
        self.act5 = self.act
        if(self.bn): self.bn5 = BatchNorm2d(self.filters[4])
        self.pool5 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout5 = Dropout(p=self.dropout)

    
        ####### Fully Connected Layer #######
        self.dense_neurons = curr_dim * curr_dim * self.filters[4]
        self.fc1 = Linear(in_features=(curr_dim * curr_dim * self.filters[4]), out_features=self.dense_neurons)  # How to calculate dimension of filters at previous level
        self.act6 = self.act
        self.dropout6 = Dropout(p=0.5)
        

        ####### Output Layer #######
        self.out = Linear(in_features=self.dense_neurons, out_features=10)
        self.act7 = LogSoftmax(dim=1)


    def getActivation(self, act):
        act = act.lower()
        if(act == 'relu'):
            return ReLU()
        elif(act == 'leakyrelu'):
            return LeakyReLU()
        elif(act == 'gelu'):
            return GELU()
        elif(act == 'selu'):
            return SELU()
        elif(act == 'mish'):
            return Mish()
    

    def filter_logic(self, filter, org):
        level = []
        org = org.lower()
        if org == 'same':
            level = [filter for i in range(5)]
        elif org == 'double':
            level = [filter*pow(2,i) for i in range(5)]
        elif org == 'half':
            level = [max(filter//pow(2,i),1) for i in range(5)]
        return level

    

    def forward(self, r):

        r=self.conv1(r)
        r=self.act1(r)
        if(self.bn): r=self.bn1(r)
        r=self.pool1(r)
        r=self.dropout1(r)

        r=self.conv2(r)
        r=self.act2(r)
        if(self.bn): r=self.bn2(r)
        r=self.pool2(r)
        r=self.dropout2(r)

        r=self.conv3(r)
        r=self.act3(r)
        if(self.bn): r=self.bn3(r)
        r=self.pool3(r)
        r=self.dropout3(r)

        r=self.conv4(r)
        r=self.act4(r)
        if(self.bn): r=self.bn4(r)
        r=self.pool4(r)
        r=self.dropout4(r)

        r=self.conv5(r)
        r=self.act5(r)
        if(self.bn): r=self.bn5(r)
        r=self.pool5(r)
        r=self.dropout5(r)

        r=flatten(r,1)
        r=self.fc1(r)
        r=self.act6(r)
        r=self.dropout6(r)
        
        r=self.out(r)
        output=self.act7(r)

        return output
        


In [ ]:
# # For personal debug use
# def train(param):
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     model = CNN(param).to(device)
#     optimizer = Adam(model.parameters(), lr=param['learning_rate'])
#     loss_function = NLLLoss()
#     train_data_loader, validation_data_loader = get_data(param, 'train')
    

#     for epo in range(param['epochs']):
#         model.train()
#         totalTrainLoss = 0
#         totalValLoss = 0
#         trainCorrect = 0
#         valCorrect = 0
#         train_counter=0
#         validation_counter=0
#         for (image, label) in train_data_loader:
#             (image, label) = (image.to(device), label.to(device))
#             prediction = model(image)
#             loss = loss_function(prediction, label)
            
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
            
#             totalTrainLoss += loss
#             trainCorrect += (prediction.argmax(1) == label).type(float).sum().item()
#             train_counter+=1
#             # print(train_counter)
        
#         with no_grad():
#             model.eval()
#             for (image, label) in validation_data_loader:
#                 (image, label) = (image.to(device), label.to(device))
#                 pred = model(image)
#                 totalValLoss += loss_function(pred, label)
#                 valCorrect += (pred.argmax(1) == label).type(float).sum().item()
#                 validation_counter+=1

#         print(f"Epochs --> {epo}")
#         print(f"Train Loss --> {(totalTrainLoss/train_counter).cpu().detach().numpy()}")
#         print(f"Train Accuracy --> {trainCorrect/len(train_data_loader.dataset)}")
#         print(f"Validation Loss --> {(totalValLoss/validation_counter).cpu().detach().numpy()}")
#         print(f"Validation Accuracy --> {valCorrect/len(validation_data_loader.dataset)}")
#         print("##########################################################################################")
    
#     return model

In [ ]:
mdl = train(PARAMETERS)

In [ ]:
# test_data_loader = get_data(PARAMETERS, 'test')
# tstCorrect = 0
# tstCounter = 0
# y = []
# y_pred = []
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# with no_grad():
#     mdl.eval()
#     for (image, label) in test_data_loader:
#         (image, label) = (image.to(device), label.to(device))
#         pred = mdl(image)
#         y.extend(label.tolist())
#         y_pred.extend(pred.argmax(1).tolist())
#         # print(pred)
#         tstCorrect += (pred.argmax(1) == label).type(float).sum().item()
#         tstCounter+=PARAMETERS['batch_size']

# print(tstCorrect)
# print(tstCounter)
# print((tstCorrect/tstCounter)*100)

In [ ]:
## for wandb sweeps
def train():
    wandb.init()
    param = wandb.config
    wandb.run.name = f'fltr_{param.filters}_fltrOrg_{param.filter_org}_dataAug_{param.data_augmentation}_batchNorm_{param.batch_normalization}_act_{param.activation}_batchSz_{param.batch_size}'

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = CNN(param).to(device)
    optimizer = Adam(model.parameters(), lr=param['learning_rate'])
    loss_function = NLLLoss()
    train_data_loader, validation_data_loader = get_data(param, 'train')
    

    for epo in range(param['epochs']):
        model.train()
        totalTrainLoss = 0
        totalValLoss = 0
        trainCorrect = 0
        valCorrect = 0
        train_counter=0
        validation_counter=0
        for (image, label) in train_data_loader:
            (image, label) = (image.to(device), label.to(device))
            prediction = model(image)
            loss = loss_function(prediction, label)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            totalTrainLoss += loss
            trainCorrect += (prediction.argmax(1) == label).type(float).sum().item()
            train_counter+=1
          
        
        with no_grad():
            model.eval()
            for (image, label) in validation_data_loader:
                (image, label) = (image.to(device), label.to(device))
                pred = model(image)
                totalValLoss += loss_function(pred, label)
                valCorrect += (pred.argmax(1) == label).type(float).sum().item()
                validation_counter+=1

        tr_ls = (totalTrainLoss/train_counter).cpu().detach().numpy()
        tr_acc = trainCorrect/len(train_data_loader.dataset)
        val_ls = (totalValLoss/validation_counter).cpu().detach().numpy()
        val_acc = valCorrect/len(validation_data_loader.dataset)
        print(f"Epoch --> {epo}")
        print(f"Train Loss --> {tr_ls}")
        print(f"Train Accuracy --> {tr_acc}")
        print(f"Validation Loss --> {val_ls}")
        print(f"Validation Accuracy --> {val_acc}")
        
        lg={
            'epoch': epo+1,
            'tr_accuracy': tr_acc,
            'val_accuracy': val_acc,
            'tr_loss': tr_ls,
            'val_loss': val_ls
        }
        wandb.log(lg)

        # return model

In [2]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [54]:
# sweep_config = {
#   "method": "bayes", 
#   "name": "Q2 Sweep",
#   "metric": {"goal": "maximize", "name": "val_accuracy"},
#   "parameters": {
#     "data_augmentation":{"values": [True,False]},  # List of boolean values
#     "batch_normalization":{"values": [True,False]},  # List of boolean values
#     "filters":{"values": [32]},  # List of filter values for first layer
#     "filter_org":{"values": ["same","half","double"]},  # List of filter organization options.. , "half", "double"
#     "dropout":{"values": [0.2,0.3]},  # Dropout rates
#     "activation":{"values": ["relu","leakyrelu","gelu","mish","selu"]},  # Activation functions... , "relu", "leaky_relu"
#     "batch_size":{"values": [32]},
#     "learning_rate":{"values": [0.001,0.0001]},
#     "epochs":{"values": [10]},
#     "dim":{"values": [256]},
#     "conv_kernel_size":{"values": [3,5]},
#     "dense_neurons":{"values": [512]},
#     "train_data_dir":{"values": ["/kaggle/working/inaturalist_12K/train"]},
#     "test_data_dir":{"values": ["/kaggle/working/inaturalist_12K/val"]}
#   }
# }


In [8]:
sweep_config = {
  "method": "bayes", 
  "name": "Q2 Sweep(modified run)",
  "metric": {"goal": "maximize", "name": "val_accuracy"},
  "parameters": {
    "data_augmentation":{"values": [True]},  # List of boolean values
    "batch_normalization":{"values": [True]},  # List of boolean values
    "filters":{"values": [32,64]},  # List of filter values for first layer
    "filter_org":{"values": ["half"]},  # List of filter organization options.. , "half", "double"
    "dropout":{"values": [0.1,0.2]},  # Dropout rates
    "activation":{"values": ["relu"]},  # Activation functions... , "relu", "leaky_relu"
    "batch_size":{"values": [32,64]},
    "learning_rate":{"values": [0.0001,0.00001]},
    "epochs":{"values": [10,15]},
    "dim":{"values": [256]},
    "conv_kernel_size":{"values": [3]},
    "dense_neurons":{"values": [128,256]},
    "train_data_dir":{"values": ["/kaggle/working/inaturalist_12K/train"]},
    "test_data_dir":{"values": ["/kaggle/working/inaturalist_12K/val"]}
  }
}


In [9]:
wandb.login(key="b8d44a4abbab8753e976a6e5ab717fd669ba99a2")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs24m030 (cs24m030-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [10]:
# wandb.init()
sweep_id = wandb.sweep(sweep_config, project="DA6401_A2")

Create sweep with ID: zoh6s0ci
Sweep URL: https://wandb.ai/cs24m030-indian-institute-of-technology-madras/DA6401_A2/sweeps/zoh6s0ci


In [ ]:
wandb.agent("3yycib0m", function=train, count=20)
wandb.finish()

wandb: Agent Starting Run: vcvmp7vy with config:
wandb: 	activation: relu
wandb: 	batch_normalization: True
wandb: 	batch_size: 64
wandb: 	conv_kernel_size: 3
wandb: 	data_augmentation: True
wandb: 	dense_neurons: 128
wandb: 	dim: 256
wandb: 	dropout: 0.1
wandb: 	epochs: 10
wandb: 	filter_org: half
wandb: 	filters: 64
wandb: 	learning_rate: 0.0001
wandb: 	test_data_dir: /kaggle/working/inaturalist_12K/val
wandb: 	train_data_dir: /kaggle/working/inaturalist_12K/train


Epoch --> 0
Train Loss --> 2.324681043624878
Train Accuracy --> 0.130125
Validation Loss --> 2.244976758956909
Validation Accuracy --> 0.18159079539769885
Epoch --> 1
Train Loss --> 2.240840435028076
Train Accuracy --> 0.170125
Validation Loss --> 2.1959280967712402
Validation Accuracy --> 0.2016008004002001
Epoch --> 2
Train Loss --> 2.184339761734009
Train Accuracy --> 0.203125
Validation Loss --> 2.1752407550811768
Validation Accuracy --> 0.19559779889944973
Epoch --> 3
Train Loss --> 2.1566975116729736
Train Accuracy --> 0.215
Validation Loss --> 2.152392864227295
Validation Accuracy --> 0.21310655327663833
Epoch --> 4
Train Loss --> 2.128690004348755
Train Accuracy --> 0.225625
Validation Loss --> 2.1398568153381348
Validation Accuracy --> 0.21560780390195097
Epoch --> 5
Train Loss --> 2.103644371032715
Train Accuracy --> 0.24425
Validation Loss --> 2.1254026889801025
Validation Accuracy --> 0.22811405702851426
Epoch --> 6
Train Loss --> 2.0898923873901367
Train Accuracy --> 0.239

epoch,▁▂▃▃▄▅▆▆▇█
tr_accuracy,▁▃▅▅▆▇▇███
tr_loss,█▆▅▄▃▃▂▂▁▁
val_accuracy,▁▃▃▅▅▆▇▇██
val_loss,█▆▅▄▃▂▂▂▁▁
epoch,10
tr_accuracy,0.26575
tr_loss,2.04325
val_accuracy,0.24012
val_loss,2.09907


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: hguvy8wn with config:
wandb: 	activation: relu
wandb: 	batch_normalization: True
wandb: 	batch_size: 64
wandb: 	conv_kernel_size: 3
wandb: 	data_augmentation: True
wandb: 	dense_neurons: 256
wandb: 	dim: 256
wandb: 	dropout: 0.1
wandb: 	epochs: 15
wandb: 	filter_org: half
wandb: 	filters: 64
wandb: 	learning_rate: 0.0001
wandb: 	test_data_dir: /kaggle/working/inaturalist_12K/val
wandb: 	train_data_dir: /kaggle/working/inaturalist_12K/train


Epoch --> 0
Train Loss --> 2.3516032695770264
Train Accuracy --> 0.1115
Validation Loss --> 2.271414279937744
Validation Accuracy --> 0.14207103551775888
Epoch --> 1
Train Loss --> 2.275578022003174
Train Accuracy --> 0.1525
Validation Loss --> 2.22133469581604
Validation Accuracy --> 0.1935967983991996
Epoch --> 2
Train Loss --> 2.208864212036133
Train Accuracy --> 0.192125
Validation Loss --> 2.15555477142334
Validation Accuracy --> 0.23711855927963982
Epoch --> 3
Train Loss --> 2.1678929328918457
Train Accuracy --> 0.20525
Validation Loss --> 2.1339824199676514
Validation Accuracy --> 0.24212106053026514
Epoch --> 4
Train Loss --> 2.15228009223938
Train Accuracy --> 0.218625
Validation Loss --> 2.102994203567505
Validation Accuracy --> 0.2621310655327664
Epoch --> 5
Train Loss --> 2.132068157196045
Train Accuracy --> 0.22675
Validation Loss --> 2.092700481414795
Validation Accuracy --> 0.26863431715857927
Epoch --> 6
Train Loss --> 2.1048240661621094
Train Accuracy --> 0.235625
Vali

...

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
tr_accuracy,▁▂▃▄▅▅▅▆▆▇▇▇███
tr_loss,█▇▆▅▅▄▄▃▃▃▂▂▂▁▁
val_accuracy,▁▂▃▄▅▅▆▇▇▇█████
val_loss,█▇▆▅▄▄▃▂▂▂▂▁▂▁▁
epoch,15
tr_accuracy,0.29387
tr_loss,1.98292
val_accuracy,0.28414
val_loss,2.00935


wandb: Agent Starting Run: oslfgc6t with config:
wandb: 	activation: relu
wandb: 	batch_normalization: True
wandb: 	batch_size: 64
wandb: 	conv_kernel_size: 3
wandb: 	data_augmentation: True
wandb: 	dense_neurons: 256
wandb: 	dim: 256
wandb: 	dropout: 0.1
wandb: 	epochs: 15
wandb: 	filter_org: half
wandb: 	filters: 64
wandb: 	learning_rate: 0.0001
wandb: 	test_data_dir: /kaggle/working/inaturalist_12K/val
wandb: 	train_data_dir: /kaggle/working/inaturalist_12K/train


Epoch --> 0
Train Loss --> 2.3345699310302734
Train Accuracy --> 0.122375
Validation Loss --> 2.236279249191284
Validation Accuracy --> 0.16808404202101052
Epoch --> 1
Train Loss --> 2.2555112838745117
Train Accuracy --> 0.157125
Validation Loss --> 2.184647798538208
Validation Accuracy --> 0.20210105052526264
Epoch --> 2
Train Loss --> 2.1988139152526855
Train Accuracy --> 0.198
Validation Loss --> 2.14717173576355
Validation Accuracy --> 0.2161080540270135
Epoch --> 3
Train Loss --> 2.163601875305176
Train Accuracy --> 0.211375
Validation Loss --> 2.121948719024658
Validation Accuracy --> 0.23861930965482742
Epoch --> 4
Train Loss --> 2.134932518005371
Train Accuracy --> 0.230125
Validation Loss --> 2.1013314723968506
Validation Accuracy --> 0.2571285642821411
Epoch --> 5
Train Loss --> 2.108485698699951
Train Accuracy --> 0.23825
Validation Loss --> 2.0855937004089355
Validation Accuracy --> 0.2511255627813907
Epoch --> 6
Train Loss --> 2.0921237468719482
Train Accuracy --> 0.2465
V

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
tr_accuracy,▁▂▄▅▅▆▆▆▆▇▇▇▇██
tr_loss,█▆▅▅▄▄▃▃▃▂▂▂▂▁▁
val_accuracy,▁▃▄▅▆▆▇▇▇▇█████
val_loss,█▇▅▅▄▄▃▃▃▂▂▁▁▁▁
epoch,15
tr_accuracy,0.29275
tr_loss,1.9768
val_accuracy,0.28814
val_loss,1.98856


wandb: Agent Starting Run: uyi00p2j with config:
wandb: 	activation: relu
wandb: 	batch_normalization: True
wandb: 	batch_size: 64
wandb: 	conv_kernel_size: 3
wandb: 	data_augmentation: True
wandb: 	dense_neurons: 256
wandb: 	dim: 256
wandb: 	dropout: 0.1
wandb: 	epochs: 15
wandb: 	filter_org: half
wandb: 	filters: 64
wandb: 	learning_rate: 0.0001
wandb: 	test_data_dir: /kaggle/working/inaturalist_12K/val
wandb: 	train_data_dir: /kaggle/working/inaturalist_12K/train
